In [2]:
import pandas as pd
import plotly.graph_objects as go

In [18]:
import pandas as pd
import plotly.graph_objects as go

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25-Apr25 Calendar_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=15).mean()

# Initialize variables
entry_points = []  # List to store entry points for visualization
exit_points = []   # List to store exit points for visualization
positions = []     # To keep track of open positions
entry_price = None
tick_size = 0.001
last_high = None  # Track last high for exit conditions
last_low = None   # Track last low for exit conditions

# Iterate over DataFrame to backtest strategy
for i in range(1, len(df)):
    # Check for crossing points (sign change in the difference)
    if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:  # MA9 crosses above MA12
        if positions and positions[-1]['type'] == 'short':  # Close short position if open
            profit = (entry_price - df['Close'][i]) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'short'))
            positions.pop()
        # Open long position
        positions.append({'type': 'long', 'entry_price': df['Close'][i]})
        entry_price = df['Close'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
        last_high = df['High'][i]  # Update last high for exit condition
        last_low = df['Low'][i]    # Update last low for exit condition

    elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:  # MA9 crosses below MA12
        if positions and positions[-1]['type'] == 'long':  # Close long position if open
            profit = (df['Close'][i] - entry_price) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'long'))
            positions.pop()
        # Open short position
        positions.append({'type': 'short', 'entry_price': df['Close'][i]})
        entry_price = df['Close'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'short'))
        last_high = df['High'][i]  # Update last high for exit condition
        last_low = df['Low'][i]    # Update last low for exit condition

    # Check for trend-breaking moves
    if positions:
        if positions[-1]['type'] == 'long' and df['High'][i] < last_high:
            profit = (df['Close'][i] - entry_price) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'long'))
            positions.pop()
            last_low = df['Low'][i]  # Update last low to current low
        elif positions[-1]['type'] == 'short' and df['High'][i] > last_high:
            profit = (entry_price - df['Close'][i]) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'short'))
            positions.pop()
            last_high = df['High'][i]  # Update last high to current high

# Calculate total profit
total_profit = sum(profit for profit in trades if profit is not None)

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA15', line=dict(color='red')))

# Add entry and exit points to the chart
for point in entry_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Entry {point[2]}", 
                             marker=dict(symbol='triangle-up' if point[2] == 'long' else 'triangle-down', color='green' if point[2] == 'long' else 'red')))
for point in exit_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Exit {point[2]}", 
                             marker=dict(symbol='cross', color='blue')))

# Update layout
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

# Print total profit
print(f"Total Profit: {total_profit} ticks")


Total Profit: -299.9999999999992 ticks


In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=12).mean()

# Initialize variables
entry_points = []
exit_points = []
positions = []
trades = []
daily_returns = []

# DataFrame to store trade details
trade_details = pd.DataFrame(columns=['Entry Date', 'Exit Date', 'Trade Type', 'Entry Price', 'Exit Price', 'Profit'])

# Constants for tick size and tick price
tick_size = 0.025
tick_price = 10

# Iterate over DataFrame to backtest strategy
for i in range(1, len(df)):
    daily_return = 0
    
    # Check for crossing points (sign change in the difference)
    if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:
        # MA9 crosses above MA12
        if positions and positions[-1]['type'] == 'long':
            # Close long position
            exit_price = df['Close'][i]
            entry_price = positions[-1]['entry_price']
            ticks = (exit_price - entry_price) / tick_size
            profit = ticks * tick_price
            exit_points.append((df['Serial'][i], exit_price, 'long'))
            trades.append(profit)
            trade_details = pd.concat([trade_details, pd.DataFrame({
                'Entry Date': [positions[-1]['entry_date']],
                'Exit Date': [df['Date'][i]],
                'Trade Type': ['Long'],
                'Entry Price': [entry_price],
                'Exit Price': [exit_price],
                'Profit': [profit]
            })], ignore_index=True)
            daily_return = profit / entry_price
            positions.pop()
        
        # Open short position
        positions.append({'type': 'short', 'entry_price': df['Close'][i], 'entry_date': df['Date'][i]})
        entry_points.append((df['Serial'][i], df['Close'][i], 'short'))
    
    elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:
        # MA9 crosses below MA12
        if positions and positions[-1]['type'] == 'short':
            # Close short position
            exit_price = df['Close'][i]
            entry_price = positions[-1]['entry_price']
            ticks = (entry_price - exit_price) / tick_size
            profit = ticks * tick_price
            exit_points.append((df['Serial'][i], exit_price, 'short'))
            trades.append(profit)
            trade_details = pd.concat([trade_details, pd.DataFrame({
                'Entry Date': [positions[-1]['entry_date']],
                'Exit Date': [df['Date'][i]],
                'Trade Type': ['Short'],
                'Entry Price': [entry_price],
                'Exit Price': [exit_price],
                'Profit': [profit]
            })], ignore_index=True)
            daily_return = profit / entry_price
            positions.pop()
        
        # Open long position
        positions.append({'type': 'long', 'entry_price': df['Close'][i], 'entry_date': df['Date'][i]})
        entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
    
    daily_returns.append(daily_return)

# Close any open position at the end of the period
if positions:
    exit_price = df['Close'].iloc[-1]
    entry_price = positions[-1]['entry_price']
    if positions[-1]['type'] == 'long':
        ticks = (exit_price - entry_price) / tick_size
    else:
        ticks = (entry_price - exit_price) / tick_size
    profit = ticks * tick_price
    exit_points.append((df['Serial'].iloc[-1], exit_price, positions[-1]['type']))
    trades.append(profit)
    trade_details = pd.concat([trade_details, pd.DataFrame({
        'Entry Date': [positions[-1]['entry_date']],
        'Exit Date': [df['Date'].iloc[-1]],
        'Trade Type': [positions[-1]['type'].capitalize()],
        'Entry Price': [entry_price],
        'Exit Price': [exit_price],
        'Profit': [profit]
    })], ignore_index=True)

# Calculate total profit
total_profit = sum(trades)

# Calculate Sharpe Ratio
risk_free_rate = 0.02  # Assume 2% annual risk-free rate
daily_rf_rate = (1 + risk_free_rate) ** (1/252) - 1
excess_returns = np.array(daily_returns) - daily_rf_rate
sharpe_ratio = np.sqrt(252) * np.mean(excess_returns) / np.std(excess_returns)

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA12', line=dict(color='red')))

# Add entry and exit points to the chart
for point in entry_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Entry {point[2]}", 
                             marker=dict(symbol='triangle-up' if point[2] == 'long' else 'triangle-down', size=10, color='green' if point[2] == 'long' else 'red')))
for point in exit_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Exit {point[2]}", 
                             marker=dict(symbol='x', size=12, color='blue')))

# Update layout
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

# Print performance metrics
print(f"Total Profit: ${total_profit:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

# Display trade details
print(trade_details)


/tmp/ipykernel_1681/2417635115.py:45: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Total Profit: $13240.00
Sharpe Ratio: 1.79
   Entry Date  Exit Date Trade Type  Entry Price  Exit Price  Profit
0  2023-09-29 2023-10-17       Long      196.025     196.375   140.0
1  2023-10-17 2023-10-20      Short      196.375     192.500  1550.0
2  2023-10-20 2023-11-06       Long      192.500     190.350  -860.0
3  2023-11-06 2023-11-10      Short      190.350     184.000  2540.0
4  2023-11-10 2023-11-24       Long      184.000     183.025  -390.0
5  2023-11-24 2023-11-28      Short      183.025     182.975    20.0
6  2023-11-28 2023-12-19       Long      182.975     182.500  -190.0
7  2023-12-19 2024-01-17      Short      182.500     185.275 -1110.0
8  2024-01-17 2024-01-22       Long      185.275     186.950   670.0
9  2024-01-22 2024-02-15      Short      186.950     189.900 -1180.0
10 2024-02-15 2024-02-28       Long      189.900     191.350   580.0
11 2024-02-28 2024-03-11      Short      191.350     192.425  -430.0
12 2024-03-11 2024-03-12       Long      192.425     192.925